# 3. nD k-class Gaussian discriminant analysis

### (a) Select a k-class dataset with continuous nD features.

### (b) Estimate the model parameters and compute a discriminant function based on the distribution in each class.

In [2]:
import pandas as pd
import numpy as np

# 3. nD k-class Gaussian discriminant analysis

# (a) Select a k-class dataset with continuous nD features.

def rf(fn):
    data = pd.read_csv(fn, sep=',', header=None, skipinitialspace=True)
    return data

## (b) Estimate the model parameters and compute a discriminant function based on the distribution in each class.

def getPrmtr(train_data,cnum):
    fnum = train_data.shape[1]-1
    m = np.zeros(cnum)
    mean = np.zeros((cnum, fnum))
    sigma = np.zeros((cnum, fnum,fnum))
    for i in range(0, cnum):
        subcls = train_data[train_data[4] == i+1]
        x=subcls.loc[:,:3]
        m[i] = len(x)
        mean[i]=x.mean()
    for j in range(0,len(train_data)):
        data = train_data.iloc[j]
        x = data[:fnum]
        y = data[4]
        temp = np.outer((x-mean[y-1]),(x-mean[y-1]))
        sigma[y-1] += temp/m[y-1]
    return (mean, sigma, m ) # a = alphalist

fn = 'iris.data'
data = rf(fn)
print 'Read iris.data with 4 features and 3 classes.'
print data.head()
temp = getPrmtr(data, 3)
print 'Means of n classes:'
print temp[0]
print 'Sigmas of n classes'
print temp[1]
print 'Alpha:'
print temp[2]

Read iris.data with 4 features and 3 classes.
     0    1    2    3  4
0  5.1  3.5  1.4  0.2  1
1  4.9  3.0  1.4  0.2  1
2  4.7  3.2  1.3  0.2  1
3  4.6  3.1  1.5  0.2  1
4  5.0  3.6  1.4  0.2  1
Means of n classes:
[[ 5.006  3.418  1.464  0.244]
 [ 5.936  2.77   4.26   1.326]
 [ 6.588  2.974  5.552  2.026]]
Sigmas of n classes
[[[ 0.121764  0.098292  0.015816  0.010336]
  [ 0.098292  0.142276  0.011448  0.011208]
  [ 0.015816  0.011448  0.029504  0.005584]
  [ 0.010336  0.011208  0.005584  0.011264]]

 [[ 0.261104  0.08348   0.17924   0.054664]
  [ 0.08348   0.0965    0.081     0.04038 ]
  [ 0.17924   0.081     0.2164    0.07164 ]
  [ 0.054664  0.04038   0.07164   0.038324]]

 [[ 0.396256  0.091888  0.297224  0.048112]
  [ 0.091888  0.101924  0.069952  0.046676]
  [ 0.297224  0.069952  0.298496  0.047848]
  [ 0.048112  0.046676  0.047848  0.073924]]]
Alpha:
[ 50.  50.  50.]


D:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
D:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


### (c) Classify the examples and measure your error. Compute the confusion matrix, precision, recall, F-measure, and accuracy.

In [6]:
# K fold
def k_fold(size, k, shuffle=False):
    index = np.arange(0, size)
    if shuffle:
        np.random.shuffle(index)
    index = np.reshape(index, (k, size / k))
    #     print index
    test = []
    train = []
    for i in range(0, k):
        test.append(index[i])
        temp = np.delete(index, i, 0)
        train.append(temp.flatten())
    return test, train


# calculate the membership for one example x
from pylab import *


def calcm(x, a, m, s):
    ds = det(s)
    ins = inv(s)
    qf = dot((x - m).T, ins)
    g = log(a) - 0.5*log(ds) - 0.5 * np.dot(qf, (x - m))
    return g


# given a test dataset value, return a list of claasifications
def classificaiton(data_test, al, ml, sl):
    x_test = data_test.loc[:, :3]
    xnum = len(x_test)
    cnum = len(al)
    prelist = []

    for i in range(0, xnum):
        ex = x_test.iloc[i]
        c = np.zeros(cnum)

        for j in range(0, cnum):
            c[j] = calcm(ex, al[j], ml[j], sl[j])
        # print "c is", c
        maxi = np.argmax(c)
        prelist.append(maxi + 1)
    return prelist


def cMatrix(testdata, prelist, cnum):
    y_test = testdata[4].tolist()
    print type(prelist)
    cm = np.zeros((cnum, cnum))
    # print len(y_test)
    for i in range(0, len(y_test)):
        rl = y_test[i]
        pre = prelist[i]
        # print pre,"test:",rl
        cm[rl - 1][pre - 1] += 1
    return cm


# Compute the confusion matrix, precision, recall,F-measure, and accuracy
def evalpf(cm):
    size = cm.shape[0]
    precision = []
    recall = []
    true = 0
    fm = []
    for i in range(0, size):
        tp = cm[i][i]
        true += tp
        sumcol = cm[:, i].sum()
        sumrow = cm[i, :].sum()
        pre = tp / sumcol
        precision.append(pre)
        rec = tp / sumrow
        recall.append(rec)
        fm.append(2 * pre * rec / (pre + rec))

    accuracy = true / cm.sum()
    return (precision, recall, fm, accuracy)


def clsmain(fn='iris.data', kfold=5, classnum=3, shuffle=True):
    data = rf(fn)
    total = len(data)
    i_test, i_train = k_fold(total, kfold, True)
    prelist = []
    data_test = data.iloc[i_test[0],]
    data_train = data.iloc[i_train[0],]
    parms = getPrmtr(data_train, classnum)  # (mean, sigma/mean, m )
    trainnum = len(data_train)
    alfa = parms[2]/trainnum
    y_predict = classificaiton(data_test, alfa, parms[0], parms[1])
    cm = cMatrix(data_test, y_predict,3)
    print 'confusion matrix is '
    print cm
    result = evalpf(cm)
    print 'result of precision, recall,F-measure, and accuracy are'
    print result


clsmain()

<type 'list'>
confusion matrix is 
[[  9.   0.   0.]
 [  0.  11.   0.]
 [  0.   0.  10.]]
result of precision, recall,F-measure, and accuracy are
([1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], 1.0)


D:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
D:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


This result shows that testing data fit the predictions perfactly.